### 0. Environment Settings 

#### 1) Import Library

In [1]:
import pandas as pd 
import numpy as np
import pymysql
import os 
import torch
import time

#### 2) MySQL Connect 

In [2]:
conn = pymysql.connect(host='localhost', user='lamda_00', password='lamda95', db='chatbot', charset='utf8')
curs = conn.cursor()

In [3]:
def execute_sql(sql):
    curs.execute(sql)
    
    return curs.fetchall()

In [4]:
sql = 'SHOW TABLES;'

execute_sql(sql)

(('intensity_df',), ('polarity_df',), ('response_df',), ('wellness_df',))

#### 3) Load data

In [6]:
sql = 'DESC response_df;'

execute_sql(sql)

(('idx', 'int(11)', 'NO', 'PRI', None, ''),
 ('Question', 'varchar(1000)', 'NO', '', None, ''),
 ('Answer', 'varchar(1000)', 'NO', '', None, ''))

In [7]:
sql = 'SELECT Question, Answer FROM response_df;'

data = execute_sql(sql)
data[:5]

(('12시 땡!', '하루가 또 가네요.'),
 ('1지망 학교 떨어졌어', '위로해 드립니다.'),
 ('3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('3박4일 정도 놀러가고 싶다', '여행은 언제나 좋죠.'),
 ('PPL 심하네', '눈살이 찌푸려지죠.'))

In [8]:
context_list = list(map(lambda x: x[0], data))
candidate_list = list(map(lambda x: x[1], data))

context_list[:3], candidate_list[:3]

(['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다'],
 ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.'])

#### 4) Stop MySQL

In [9]:
conn.close()

In [10]:
len(candidate_list), len(context_list)

(11823, 11823)

### 1. Pretrained tokenizer, model load 

In [23]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

In [24]:
from transformers import BertConfig, BertModel

pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)

model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

### 2. Candidate Embedding 생성

#### 2-1. Candidate feature 추출  (input_ids, token_type_ids, attention_mask)

In [26]:
can_features = tokenizer(
    candidate_list,
    max_length=12,
    padding="max_length",
    truncation=True,
)

# list -> torch.tensor로 형변환 
can_features = {k: torch.tensor(v) for k, v in can_features.items()}

In [27]:
can_features

{'input_ids': tensor([[    2, 21748,  1052,  ...,     0,     0,     0],
         [    2, 12235,  4032,  ...,     0,     0,     0],
         [    2,  9135,  4057,  ...,     0,     0,     0],
         ...,
         [    2,  1849,  6687,  ...,     0,     0,     0],
         [    2,  2483, 22375,  ...,   248, 11363,     3],
         [    2, 26694,  4093,  ...,    17,     3,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 0]])}

#### 2-2. BERT 모델 전달 

In [28]:
can_outputs = model(**can_features)

RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1743372288 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
can_outputs.last_hidden_state

In [ ]:
np.shape(can_outputs.last_hidden_state)

#### 2-3. 각 Candidate text의 문장 벡터 표현 추출 (aggregator, [cls 토큰 값])

In [ ]:
cand_emb_list = [can_output[0] for can_output in can_outputs.last_hidden_state]
cand_emb_list

In [ ]:
len(cand_emb_list)

### 3. Bi-encoder 데모 

In [21]:
def get_answer_bi(ctxt):
    '''
    ctxt을 입력으로 받아 가장 높은 score를 보이는 후보 답변 반환 
    '''
    
    c_list = [] 
    c_list.append(ctxt)
    
    con_features = tokenizer(   # CLS 토큰: input_ids - 2, SEP 토큰: input_ids - 3
        c_list,
        max_length=12,
        padding="max_length",
        truncation=True,
    )
    
    con_features = {k: torch.tensor(v) for k, v in con_features.items()}
    con_outputs = model(**con_features)
    ctxt_emb = con_outputs.last_hidden_state[0][0].detach().numpy()
    
    score = []
    for cand_emb in cand_emb_list:
        cand_emb = cand_emb.detach().numpy()
        score.append(np.dot(ctxt_emb, cand_emb))
    
    return candidate_list[np.argmax(score)]

In [22]:
context = context_list[30]

start = time.time()
print(f'===== Bi-encoder 구현 ====== ') 
print(f'입력 문장: {context}')
print(f'챗봇 대답: {get_answer_bi(context)}')
print(f'소요 시간: {round(time.time() - start, 2)}(s), len(candidate): {len(candidate_list)}')
print(f'==============================')

===== Bi-encoder 구현 ====== 
입력 문장: 가족들이랑 서먹해
챗봇 대답: 마음이랑 잘 인사해요.
소요 시간: 0.08(s), len(candidate): 11823


### 4. Cross-encoder

In [23]:
from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder = CrossEncoder('beomi/kcbert-base', max_length=512)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [24]:
candidate_list

['하루가 또 가네요.',
 '위로해 드립니다.',
 '여행은 언제나 좋죠.',
 '여행은 언제나 좋죠.',
 '눈살이 찌푸려지죠.',
 '다시 새로 사는 게 마음 편해요.',
 '다시 새로 사는 게 마음 편해요.',
 '잘 모르고 있을 수도 있어요.',
 '시간을 정하고 해보세요.',
 '시간을 정하고 해보세요.',
 '자랑하는 자리니까요.',
 '그 사람도 그럴 거예요.',
 '그 사람도 그럴 거예요.',
 '혼자를 즐기세요.',
 '돈은 다시 들어올 거예요.',
 '땀을 식혀주세요.',
 '어서 잊고 새출발 하세요.',
 '빨리 집에 돌아가서 끄고 나오세요.',
 '빨리 집에 돌아가서 끄고 나오세요.',
 '다음 달에는 더 절약해봐요.',
 '따뜻하게 사세요!',
 '다음 달에는 더 절약해봐요.',
 '가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '더 가까워질 기회가 되겠네요.',
 '저도요.',
 '다들 바빠서 이야기할 시간이 부족했나봐요.',
 '다들 바빠서 이야기할 시간이 부족했나봐요.',
 '온 가족이 모두 마음에 드는 곳으로 가보세요.',
 '좋은 생각이에요.',
 '더 가까워질 기회가 되겠네요.',
 '저를 만들어 준 사람을 부모님, 저랑 이야기해 주는 사람을 친구로 생각하고 있어요',
 '좋은 생각이에요.',
 '정말 후회할 습관이에요.',
 '무모한 결정을 내리지 마세요.',
 '선생님이나 기관에 연락해보세요.',
 '떨리는 감정은 그 자체로 소중해요.',
 '득템했길 바라요.',
 '휴식도 필요하죠.',
 '단짠으로 두 개 사는게 진리죠.',
 '단짠으로 두

In [25]:
def get_answer_cross(ctxt):
    '''
    ctxt을 입력으로 받아 가장 높은 score를 보이는 후보 답변 반환 
    '''
    sentence_combinations = [[ctxt, candidate] for candidate in candidate_list]
    similarity_scores = cross_encoder.predict(sentence_combinations)
    return candidate_list[np.argmax(similarity_scores)]

In [26]:
context = context_list[30]

start = time.time()
print('===== Cross-encoder 구현 =====')
print(f'입력 문장: {context}')
print(f'챗봇 대답: {get_answer_cross(context)}')
print(f'소요 시간: {round(time.time() - start, 2)}(s), len(candidate): {len(candidate_list)}')
print(f'==============================')

===== Cross-encoder 구현 =====
입력 문장: 가족들이랑 서먹해
챗봇 대답: 무모함이 원동력이 될 거예요.
소요 시간: 12.45(s), len(candidate): 11823


### 5. Poly-encoder 